CS524: Introduction to Optimization Lecture 20
======================================

## Michael Ferris<br> Computer Sciences Department <br> University of Wisconsin-Madison

## October 20, 2023
--------------

In [1]:
%load_ext gams.magic

#  Production planning (Gandhi on steroids)

An engineering plant can produce five types of products: $p_1, p_2, \ldots p_5$ by using two production processes: grinding and drilling.  Each product requires the following number of hours of each process, and contributes the following amount (in hundreds of dollars) to the net total profit (see GAMS data below).

- Each unit of each product takes 20 manhours for final assembly.
- The factory has three grinding machines and two drilling machines.
- The factory works a six day week with two shifts of 8 hours/day.  
- Eight workers are employed in assembly, each working one shift per day.

In [2]:
m = gams.exchange_container
P = m.addSet('P', description='Products', records=["p" + str(i) for i in range(1,6)])
x = m.addVariable('x','positive',[P])
z = m.addVariable('z','binary',[P])

In [3]:
%%gams
sets
    R   Resources   /grinding, drilling, assembly/;

* e.g. 288 = 3 grinders * 6 days * 2 shifts * 8 hours
parameters
    b(R)    "Amount of resource R"  /grinding 288, drilling 192, assembly 384 /,
    c(P)    "Profit from producing P" /p1 55, p2 60, p3 35, p4 40, p5 20/;

table a(R,P)  "How many R required to make one P"
            p1      p2      p3      p4      p5
grinding    12      20      0       25      15
drilling    10      8       16      0       0
assembly    20      20      20      20      20;

variables profit 'profit per week';

equations defobj, rcons(R);

defobj..
  profit =e= sum(P, c(P)*x(P));

rcons(R)..
  sum(P, a(R,P)*x(P)) =l= b(R);

model ppporig /defobj, rcons/;
solve ppporig using lp max profit;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),1092.0,4,6,LP,CPLEX,0.028


In [4]:
display(x.records)

,P,level,marginal,lower,upper,scale
0,p1,12.0,0.000,0.0,inf,1.0
1,p2,7.2,0.000,0.0,inf,1.0
2,p3,0.0,-12.500,0.0,inf,1.0
3,p4,0.0,-23.125,0.0,inf,1.0
4,p5,0.0,-36.875,0.0,inf,1.0


- Suppose we wish to add the constraint that we wish to make at most two products.
- At most two of the five $x_j$ can be positive.
- $z_j = \left\{ \begin{array}{cl} 1 & \mbox{make product } j\\
                                 0 & \mbox{otherwise} \\ \end{array} \right.$
- Set the relative optimality tolerance (gap) to 0, then solve bigM problem:

In [5]:
%%gams
parameter M(P);
M(P) = 100;

equation indcon(P), bound2;

indcon(P)..
  x(P) =l= M(P)*z(P);

bound2..
  sum(P, z(P)) =l= 2;

model ppp /ppporig, indcon, bound2/;
ppp.optcr = 0;
x.l(P) = 0; z.l(P) = 0;
solve ppp using mip max profit;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),1092.0,10,11,MIP,CPLEX,0.035


In [6]:
display(x.records,z.records)

,P,level,marginal,lower,upper,scale
0,p1,12.0,0.000,0.0,inf,1.0
1,p2,7.2,0.000,0.0,inf,1.0
2,p3,0.0,-12.500,0.0,inf,1.0
3,p4,0.0,-23.125,0.0,inf,1.0
4,p5,0.0,-36.875,0.0,inf,1.0


,P,level,marginal,lower,upper,scale
0,p1,1.0,-0.0,0.0,1.0,1.0
1,p2,1.0,-0.0,0.0,1.0,1.0
2,p3,0.0,-0.0,0.0,1.0,1.0
3,p4,0.0,-0.0,0.0,1.0,1.0
4,p5,0.0,-0.0,0.0,1.0,1.0


In [7]:
%%gams
equation bound12;

bound12..
  z('p1') + z('p2') =l= 1;

model ppp12 /ppporig, indcon, bound2, bound12/;
ppp12.optcr = 0;
solve ppp12 using mip max profit;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),1056.0,11,11,MIP,CPLEX,0.01


In [8]:
display(x.records,z.records)

,P,level,marginal,lower,upper,scale
0,p1,19.2,0.0,0.0,inf,1.0
1,p2,0.0,0.0,0.0,inf,1.0
2,p3,0.0,-20.0,0.0,inf,1.0
3,p4,0.0,-15.0,0.0,inf,1.0
4,p5,0.0,-35.0,0.0,inf,1.0


,P,level,marginal,lower,upper,scale
0,p1,1.0,-0.0,0.0,1.0,1.0
1,p2,0.0,500.0,0.0,1.0,1.0
2,p3,0.0,-0.0,0.0,1.0,1.0
3,p4,1.0,-0.0,0.0,1.0,1.0
4,p5,0.0,-0.0,0.0,1.0,1.0


Add minimum production amount constraint

In [9]:
%%gams
parameter u(P)    "minimum number of P to make" / set.P 8/;
equation bound5(P);

bound5(P)..
   x(P) =g= u(P)*z(P);

model pppalt /ppporig, indcon, bound5/;
pppalt.optcr = 0;
x.l(P) = 0; z.l(P) = 0;
solve pppalt using mip max profit;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),1066.6667,14,11,MIP,CPLEX,0.009


In [10]:
display(x.records,z.records)

,P,level,marginal,lower,upper,scale
0,p1,10.666667,0.000000,0.0,inf,1.0
1,p2,8.000000,0.000000,0.0,inf,1.0
2,p3,0.000000,0.000000,0.0,inf,1.0
3,p4,0.000000,-74.583333,0.0,inf,1.0
4,p5,0.000000,-48.750000,0.0,inf,1.0


,P,level,marginal,lower,upper,scale
0,p1,1.0,-0.000000,0.0,1.0,1.0
1,p2,1.0,-253.333333,0.0,1.0,1.0
2,p3,0.0,3500.000000,0.0,1.0,1.0
3,p4,0.0,-0.000000,0.0,1.0,1.0
4,p5,0.0,-0.000000,0.0,1.0,1.0


Modify problem to tighten the big M!

In [11]:
%gams M(P) = smin(R$(a(R,P) gt 0), b(R)/a(R,P)); x.l(P) = 0; z.l(P) = 0; solve pppalt using mip max profit;
display(x.records,z.records)

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),1066.6667,14,11,MIP,CPLEX,0.004


,P,level,marginal,lower,upper,scale
0,p1,10.666667,0.000000,0.0,inf,1.0
1,p2,8.000000,0.000000,0.0,inf,1.0
2,p3,0.000000,0.000000,0.0,inf,1.0
3,p4,0.000000,-74.583333,0.0,inf,1.0
4,p5,0.000000,-48.750000,0.0,inf,1.0


,P,level,marginal,lower,upper,scale
0,p1,1.0,-0.000000,0.0,1.0,1.0
1,p2,1.0,-253.333333,0.0,1.0,1.0
2,p3,0.0,420.000000,0.0,1.0,1.0
3,p4,0.0,-0.000000,0.0,1.0,1.0
4,p5,0.0,-0.000000,0.0,1.0,1.0


In [12]:
%gams_cleanup --closedown